## Wstęp

Podczas eksploracyjnej analizy danych (EDA), zauważyłem wysoką średnią ocen produktów (4.8 na 5). Postanowiłem bliżej przyjrzeć się ocenom użytkowników i spróbować ocenić na ile oddają prawdziwy poziom satysfakcji klientów. W tym celu pobrałem dodatkowe dane - cząstkowe oceny użytkowników dla każdego z ocenionych produktów. W kategorii "Mężczyzna" (po odjęciu na wcześniejszych etapach produktów niebędących kosmetykami) znajduje się 891 produktów posiadających przynajmniej jedną ocenę. W poprzednim etapie uzupełniłem dane ocen użytkowników danymi o ocenianych produktach.

In [146]:
import pandas as pd

In [147]:
reviews = pd.read_pickle('../data/reviews_extended.pkl')
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10873 entries, 0 to 10872
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   rating              10873 non-null  float64       
 1   author              10873 non-null  object        
 2   date                10873 non-null  datetime64[ns]
 3   content             10873 non-null  object        
 4   product_id          10873 non-null  int64         
 5   name                10873 non-null  string        
 6   brand               10873 non-null  object        
 7   current_price_pln   10873 non-null  float64       
 8   discount_pct        1960 non-null   float64       
 9   top_level_category  10873 non-null  string        
dtypes: datetime64[ns](1), float64(3), int64(1), object(3), string(2)
memory usage: 849.6+ KB


Zacznę od ujednolicenia typów kolumn. Kolumny tekstowe pochodzące z danych o produktach posiadają typ string, kolumny danych o ocenach są typu object.

In [148]:
reviews['author'] = reviews['author'].astype(pd.StringDtype())
reviews['content'] = reviews['content'].astype(pd.StringDtype())
reviews['brand'] = reviews['brand'].astype(pd.StringDtype())

reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10873 entries, 0 to 10872
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   rating              10873 non-null  float64       
 1   author              10873 non-null  string        
 2   date                10873 non-null  datetime64[ns]
 3   content             10873 non-null  string        
 4   product_id          10873 non-null  int64         
 5   name                10873 non-null  string        
 6   brand               10873 non-null  string        
 7   current_price_pln   10873 non-null  float64       
 8   discount_pct        1960 non-null   float64       
 9   top_level_category  10873 non-null  string        
dtypes: datetime64[ns](1), float64(3), int64(1), string(5)
memory usage: 849.6 KB


#### Imiona użytkowników

W pierwszym kroku chciałbym przyjrzeć się imionom użytkowników. Czy rozkłąd imion użytkowników jest podobny do rozkładu imion w populacji?

In [178]:
pd.set_option('display.max_rows', None)
reviews['author'] = reviews['author'].str.strip().str.upper()
names_from_reviews = reviews['author'].value_counts().reset_index()
names_from_reviews.reset_index().head(10)

,index,author,count
0,0,ANNA,366
1,1,AGNIESZKA,262
2,2,KATARZYNA,251
3,3,MONIKA,238
4,4,MAŁGORZATA,230
5,5,EWA,214
6,6,JOANNA,201
7,7,PIOTR,191
8,8,ANDRZEJ,174
9,9,KRZYSZTOF,172


Pierwszym ciekawym wnioskiem jest fakt, że produkty w kategorii "Mężczyzna" komentowane są głównie przez kobiety. Kategoria "Mężczyzna" jest swego rodzaju "meta-kategorią" albo tagiem - w tej kategorii znajdują się produkty przeznaczone nie tylko dla mężczyzn. To może tłumaczyć dlaczego tak wiele komentujących to kobiety.

Aby porównać imiona w bazie do imion w populacji ogólnej skorzystam z wykazów imion w rejestrze PESEL dostępnych na dane.gov.pl

In [150]:
names_m = pd.read_csv('../data/wykaz_imion_meskich_pesel.csv')
names_m.index.name = 'order'
names_m = names_m.reset_index()
names_m['name'] = names_m['IMIĘ_PIERWSZE']
names_m['sex'] = names_m['PŁEĆ']
names_m['count'] = names_m['LICZBA_WYSTĄPIEŃ']
names_m.drop(columns = ['IMIĘ_PIERWSZE', 'PŁEĆ', 'LICZBA_WYSTĄPIEŃ'], inplace = True)
names_m.head(10)

,order,name,sex,count
0,0,JAN,MĘŻCZYZNA,1100993
1,1,STANISŁAW,MĘŻCZYZNA,957337
2,2,PIOTR,MĘŻCZYZNA,818368
3,3,ANDRZEJ,MĘŻCZYZNA,744586
4,4,KRZYSZTOF,MĘŻCZYZNA,737638
5,5,JÓZEF,MĘŻCZYZNA,691043
6,6,TOMASZ,MĘŻCZYZNA,583316
7,7,PAWEŁ,MĘŻCZYZNA,574879
8,8,MICHAŁ,MĘŻCZYZNA,568191
9,9,ADAM,MĘŻCZYZNA,501999


In [151]:
names_f = pd.read_csv('../data/wykaz_imion_zeskich_pesel.csv')
names_f.index.name = 'order'
names_f = names_f.reset_index()
names_f['name'] = names_f['IMIĘ_PIERWSZE']
names_f['sex'] = names_f['PŁEĆ']
names_f['count'] = names_f['LICZBA_WYSTĄPIEŃ']
names_f.drop(columns = ['IMIĘ_PIERWSZE', 'PŁEĆ', 'LICZBA_WYSTĄPIEŃ'], inplace = True)
names_f.head(10)

,order,name,sex,count
0,0,ANNA,KOBIETA,1369530
1,1,MARIA,KOBIETA,1017067
2,2,ZOFIA,KOBIETA,727259
3,3,KATARZYNA,KOBIETA,681973
4,4,MAŁGORZATA,KOBIETA,626378
5,5,KRYSTYNA,KOBIETA,607456
6,6,BARBARA,KOBIETA,599896
7,7,AGNIESZKA,KOBIETA,595626
8,8,JANINA,KOBIETA,579979
9,9,EWA,KOBIETA,538622


Połącze dane imion kobiet i mężczyzn zachowując ich indeksy w kolumnie order.

In [152]:
names_joined = pd.concat([names_m, names_f], ignore_index = True)
names_joined.sort_values('count', ascending = False).reset_index(drop = True).head(5)

,order,name,sex,count
0,0,ANNA,KOBIETA,1369530
1,0,JAN,MĘŻCZYZNA,1100993
2,1,MARIA,KOBIETA,1017067
3,1,STANISŁAW,MĘŻCZYZNA,957337
4,2,PIOTR,MĘŻCZYZNA,818368


Następnie połącze dwa zbiory danych, by przyporządkować imionom użytkowników strony kolejność w bazie imion PESEL.

In [153]:
names_merged = names_from_reviews.merge(
    names_joined,
    how = "left",
    left_on = "author",
    right_on = "name"
)
names_merged['order'] = names_merged['order'].astype('Int64')
names_merged = names_merged[names_merged['order'] < 100][['name', 'order', 'sex']].reset_index(drop = True)


Porónanie imion kobiet w komentarzach do popularności imienia w bazie PESEL.

In [166]:
names_merged_f = names_merged[names_merged['sex'] == 'KOBIETA'].reset_index(drop = True).reset_index()
names_merged_f['order_diff'] = names_merged_f['order'] - names_merged_f['index']
names_merged_f[['name', 'order', 'order_diff']].head(30)

,name,order,order_diff
0,ANNA,0,0
1,AGNIESZKA,7,6
2,KATARZYNA,3,1
3,MONIKA,19,16
4,MAŁGORZATA,4,0
5,EWA,9,4
6,JOANNA,16,10
7,BEATA,28,21
8,IWONA,34,26
9,MARTA,24,15


10 najbardziej nadreprezentowanych imion żeńskich

In [176]:
names_merged_f.sort_values('order_diff', ascending = False)[['name', 'order', 'order_diff']].head(10)

,name,order,order_diff
24,EDYTA,69,45
50,WIOLETTA,92,42
61,ANGELIKA,97,36
26,MARZENA,61,35
20,ANETA,54,34
49,MARIOLA,81,32
14,SYLWIA,45,31
43,KAMILA,70,27
52,DARIA,79,27
72,ROZALIA,99,27


Z powyższej tabeli możemy wyczytać 10 najbardziej nadreprezentowanych imion żeńskich w populacji użytkowników komentujących kosmetyki w kategorii "Mężczyzna" na stronie rossmann.pl, w stosunku do popularności imion żeńskich w populacji ogólnej. Imię "Edyta", 24 najpopularniejsze imię wśród komentujących, w populacji ogólnej jest na 69 pozycji najpopularniejszych imion. Imię "Aneta", 20 najpopularniejsze imię wśród komentujących kobiet jest 54 na liście najpopularniejszych imion w bazie PESEL.

Przyczyną takich różnic może być np. wiek użytkowników serwisu. Baza PESEL zawiera wszystkie imiona, także imiona osób zmarłych. Baza PESEL zostałą uruchomiona w pełni w roku 1984. Uwzględnia więc wszystkie osoby, które żyły w roku 1984 i później.

10 najbardziej niedoreprezentowanych imion kobiecych.

In [175]:
names_merged_f.sort_values('order_diff')[['name', 'order', 'order_diff']].head(10)

,name,order,order_diff
85,MARIANNA,15,-70
79,STANISŁAWA,21,-58
53,ZOFIA,2,-51
62,HELENA,11,-51
96,JÓZEFA,47,-49
55,JANINA,8,-47
90,GENOWEFA,49,-41
87,STEFANIA,48,-39
86,KAZIMIERA,55,-31
63,ZUZANNA,35,-28


Imiona najbardziej niedoreprezentowane to imiona niegdyś bardzo popularne (Marianna to imie nadawane niegdyś zamiast imienia Maria), obecnie mniej popularne.

In [180]:
names_merged_m = names_merged[names_merged['sex'] == 'MĘŻCZYZNA'].reset_index(drop = True).reset_index()
names_merged_m['order_diff'] = names_merged_m['order'] - names_merged_m['index']
names_merged_m[['name', 'order', 'order_diff']].head(30)

,name,order,order_diff
0,PIOTR,2,2
1,ANDRZEJ,3,2
2,KRZYSZTOF,4,2
3,MAREK,12,9
4,PAWEŁ,7,3
5,TOMASZ,6,1
6,MARCIN,11,5
7,ROBERT,29,22
8,MARIUSZ,28,20
9,GRZEGORZ,15,6


10 najbardziej nadreprezentoawnych imion męskich.

In [181]:
names_merged_m.sort_values('order_diff', ascending = False)[['name', 'order', 'order_diff']].head(10)

,name,order,order_diff
43,BOGUSŁAW,81,38
24,LESZEK,60,36
40,RADOSŁAW,71,31
37,ARKADIUSZ,65,28
35,WALDEMAR,63,28
53,WITOLD,80,27
29,DANIEL,55,26
21,ARTUR,46,25
45,KRYSTIAN,70,25
28,SEBASTIAN,52,24


10 najbardziej niedoreprezentowanych imion męskich.

In [183]:
names_merged_m.sort_values('order_diff')[['name', 'order', 'order_diff']].head(10)

,name,order,order_diff
93,WŁADYSŁAW,25,-68
63,JÓZEF,5,-58
77,FRANCISZEK,27,-50
67,KAZIMIERZ,20,-47
72,ANTONI,26,-46
84,MIECZYSŁAW,41,-43
74,EDWARD,40,-34
47,HENRYK,16,-31
78,STEFAN,49,-29
88,MIKOŁAJ,61,-27


**Czego moglibyśmy dowiedzieć się na podstawie rozkładu imion?**

Rozkładu imion w bazie można by użyć do oceny średniego wieku użytkownika strony. Gdybym miał dostęp do najpopularniejszych imion nadawanych w Polsce w kolejnych latach, mógłbym porównać profil imion użytkowników do profili imion nadawanych dzieciom. Łatwo dostępne dane można znaleźć z ostatnich kilku lat, wcześniejsze dane trzebaby wyciągnąć z roczników statystycznych. Jest to ciekawy pomysł na kolejny projekt.